In [2]:
from google.colab import files
uploaded = files.upload()

Saving adult.csv to adult.csv
Saving adult.test.csv to adult.test.csv


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras import layers, optimizers, models
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers

In [3]:
train_data=pd.read_csv(r"adult.csv", encoding="UTF-8",header=None,  skiprows=1)
test_data=pd.read_csv(r"adult.test.csv", encoding="UTF-8",header=None,   skiprows=1)

In [4]:
train_data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,State-gov,77516,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K,Bachelors
1,50,Self-emp-not-inc,83311,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K,Bachelors
2,38,Private,215646,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K,HS-grad
3,53,Private,234721,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K,11th
4,28,Private,338409,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K,Bachelors
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K,Assoc-acdm
32557,40,Private,154374,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K,HS-grad
32558,58,Private,151910,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K,HS-grad
32559,22,Private,201490,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K,HS-grad


In [5]:
train_data=train_data.replace(' ?', np.nan)
test_data=test_data.replace(' ?', np.nan)
train_data=train_data.dropna()
test_data=test_data.dropna()

In [6]:
train_data.columns=['age',
              'workclass',
              'fnlwgt',
              'education_num',
              'marital_status',
              'occupation',
              'relationship',
              'race',
              'sex',
              'capital_gain',
              'capital_loss',
              'hours_per_week',
              'native_country',
              'salary',
             'education',
]

In [7]:
test_data.columns=['age',
              'workclass',
              'fnlwgt',
              'education_num',
              'marital_status',
              'occupation',
              'relationship',
              'race',
              'sex',
              'capital_gain',
              'capital_loss',
              'hours_per_week',
              'native_country',
              'salary',
             'education',
]

In [8]:
train_data=train_data.drop('fnlwgt', axis=1)
test_data=test_data.drop('fnlwgt', axis=1)

In [9]:
cat_col = train_data.select_dtypes(['object']).columns
train_data[cat_col] = train_data[cat_col].apply(lambda x: x.astype('category'))
train_data[cat_col] = train_data[cat_col].apply(lambda x: x.cat.codes)

In [10]:
cat_col = test_data.select_dtypes(['object']).columns
test_data[cat_col] = test_data[cat_col].apply(lambda x: x.astype('category'))
test_data[cat_col] = test_data[cat_col].apply(lambda x: x.cat.codes)

In [11]:
x_train=train_data.iloc[:,:-1]
y_train=train_data.iloc[:,-1:]
x_test=test_data.iloc[:,:-1]
y_test=test_data.iloc[:,-1:]

In [12]:
x_train=x_train.to_numpy()
y_train=y_train.to_numpy()
x_test=x_test.to_numpy()
y_test=y_test.to_numpy()

In [13]:
from keras.layers.normalization import BatchNormalization
from keras.layers import Dropout
from keras.optimizers import Adam
model = models.Sequential()
model.add(layers.Dense(50, activation="relu",input_dim=(13)))
model.add(layers.Dense(50, activation="relu"))
model.add(layers.Dense(16, activation='softmax'))

model.compile(loss="sparse_categorical_crossentropy" , optimizer='adam', metrics=["accuracy"])
history = model.fit(x_train, y_train, validation_split=0.1, epochs=20, batch_size=10)

Epoch 1/20
2715/2715 [==============================] - 5s 2ms/step - loss: 24.0781 - accuracy: 0.4130 - val_loss: 3.0168 - val_accuracy: 0.5794
Epoch 2/20
2715/2715 [==============================] - 4s 1ms/step - loss: 1.7008 - accuracy: 0.6622 - val_loss: 0.9743 - val_accuracy: 0.7242
Epoch 3/20
2715/2715 [==============================] - 4s 1ms/step - loss: 0.8772 - accuracy: 0.7554 - val_loss: 0.7311 - val_accuracy: 0.7693
Epoch 4/20
2715/2715 [==============================] - 4s 1ms/step - loss: 0.6012 - accuracy: 0.8137 - val_loss: 0.5908 - val_accuracy: 0.8111
Epoch 5/20
2715/2715 [==============================] - 4s 1ms/step - loss: 0.5427 - accuracy: 0.8279 - val_loss: 0.5352 - val_accuracy: 0.8247
Epoch 6/20
2715/2715 [==============================] - 4s 1ms/step - loss: 0.5082 - accuracy: 0.8370 - val_loss: 0.5200 - val_accuracy: 0.8369
Epoch 7/20
2715/2715 [==============================] - 4s 1ms/step - loss: 0.4763 - accuracy: 0.8463 - val_loss: 0.4979 - val_accuracy

In [14]:
score=model.evaluate(x_train,y_train)
print(score)

943/943 [==============================] - 1s 958us/step - loss: 0.3959 - accuracy: 0.8628
[0.3958755135536194, 0.8628406524658203]


In [15]:
encoder = LabelEncoder() 
encoded_y = encoder.fit_transform(y_test)
dummy_y = keras.utils.to_categorical(encoded_y, 16) # 001 100 010

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [16]:
from sklearn.metrics import classification_report
y_pred=model.predict(x_test)
y_pred=(y_pred>0.5)
print(classification_report(dummy_y,y_pred))

              precision    recall  f1-score   support

           0       0.93      0.93      0.93       403
           1       0.98      0.88      0.93       571
           2       1.00      0.86      0.93       200
           3       0.73      0.93      0.81        71
           4       0.98      0.91      0.94       161
           5       0.99      0.89      0.94       266
           6       0.99      0.90      0.94       221
           7       0.96      0.81      0.88       499
           8       0.99      0.80      0.89       652
           9       0.85      0.80      0.82      2526
          10       0.97      0.51      0.67       169
          11       0.98      0.90      0.94      4943
          12       0.98      0.39      0.56       887
          13       1.00      0.04      0.07        27
          14       1.00      0.32      0.48       243
          15       0.98      0.87      0.92      3221

   micro avg       0.96      0.82      0.88     15060
   macro avg       0.96   

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
